In [4]:
import sys
import subprocess
import importlib
import os
import time
import struct
import socket
import tkinter as tk
from tkinter import simpledialog, filedialog, messagebox

# ---------------------------------------------------------
# 1. 패키지 자동 설치 함수
# ---------------------------------------------------------
def install_package(module_name, package_name=None):
    if package_name is None:
        package_name = module_name
    try:
        importlib.import_module(module_name)
    except ImportError:
        print(f"Installing {package_name} ...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
            print(f"{package_name} installation completed")
        except subprocess.CalledProcessError as e:
            print(f"{package_name} installation failed (exit code {e.returncode})")
            sys.exit(1)

print("Checking required packages...")
install_package("h5py")
install_package("numpy")
install_package("cv2", "opencv-python")
install_package("PIL", "Pillow")
install_package("tensorflow")

import h5py
import cv2
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf
from tensorflow.keras.models import load_model

# ---------------------------------------------------------
# 2. 설정 및 연결 (IP 입력 팝업 추가)
# ---------------------------------------------------------
HOST_CAM = '192.168.0.60' # 기본 IP
PORT_CAM = 80
PORT_MOT = 81

client_cam = None
client_mot = None

# Tkinter 루트 윈도우 생성 (숨김 상태)
root = tk.Tk()
root.withdraw() 

while True:
    try:
        print(f"Connecting to ESP32-CAM at {HOST_CAM}...")
        
        # 소켓 생성
        client_cam = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_mot = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        # [중요] 딜레이 방지 옵션
        client_mot.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
        
        # 타임아웃 설정 (3초)
        client_cam.settimeout(3)
        client_mot.settimeout(3)
        
        client_cam.connect((HOST_CAM, PORT_CAM))
        client_mot.connect((HOST_CAM, PORT_MOT))
        
        # 연결 성공 시 타임아웃 해제
        client_cam.settimeout(None)
        client_mot.settimeout(None)
        
        print("Connected successfully!")
        break 

    except Exception as e:
        print(f"Connection Failed: {e}")
        
        # IP 입력 팝업창
        new_ip = simpledialog.askstring("Connection Failed", 
                                      f"Failed to connect to {HOST_CAM}.\n\nEnter ESP32 IP Address:",
                                      parent=root,
                                      initialvalue=HOST_CAM)
        
        if new_ip:
            HOST_CAM = new_ip # IP 업데이트 후 재시도
        else:
            print("User cancelled connection.")
            sys.exit()

# ---------------------------------------------------------
# 3. 모델 파일 선택 및 호환성 패치
# ---------------------------------------------------------
model_filename = 'keras_model.h5'

# 파일이 없으면 선택창 띄우기
if not os.path.exists(model_filename):
    print(f"'{model_filename}' not found. Please select a model file.")
    
    root.lift()
    root.attributes('-topmost', True)
    
    model_filename = filedialog.askopenfilename(title="Select Teachable Machine Model File",
                                            filetypes=[("H5 Files", "*.h5"), ("All Files", "*.*")])
    
    root.attributes('-topmost', False)

    if not model_filename:
        print("No model selected. Exiting...")
        sys.exit()

# [Teachable Machine 전용] 모델 호환성 패치 (Tensorflow 버전에 따른 버그 수정)
print(f"Checking compatibility for: {model_filename}")
try:
    f = h5py.File(model_filename, mode="r+")
    model_config_string = f.attrs.get("model_config")
    
    if isinstance(model_config_string, bytes):
        model_config_string = model_config_string.decode('utf-8')

    if model_config_string.find('"groups": 1,') != -1:
        print("Patching model config (removing 'groups': 1)...")
        model_config_string = model_config_string.replace('"groups": 1,', '')
        f.attrs.modify('model_config', model_config_string)
        f.flush()
        print("Model patched successfully.")
    f.close()
except Exception as e:
    print(f"Warning during model patch: {e}")

# ---------------------------------------------------------
# 4. 모델 로드
# ---------------------------------------------------------
print("Loading Keras model...")
try:
    model = load_model(model_filename, compile=False)
    print("Model loaded successfully!")
except Exception as e:
    messagebox.showerror("Error", f"Failed to load model:\n{e}")
    sys.exit()

# Tkinter 종료
root.destroy()

# 클래스 이름 정의
names = ['_0_forward', '_1_right', '_2_left', '_3_stop']

# [수정] 통신 프로토콜 매핑 (F, R, L, S)
cmd_chars = [b'F', b'R', b'L', b'S']

# ---------------------------------------------------------
# 5. 메인 루프 (영상 수신 -> 추론 -> 제어)
# ---------------------------------------------------------
t_now = time.time()
t_prev = time.time()
cnt_frame = 0

# Teachable Machine 입력 크기 (224x224)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

try:
    while True:
        # 1. 영상 요청
        cmd = 12
        try:
            client_cam.sendall(struct.pack('B', cmd))
        except Exception:
            print("Connection lost (Send)")
            break

        # 2. 데이터 길이 수신
        try:
            data_len_bytes = client_cam.recv(4)
            if not data_len_bytes: break
            data_len = struct.unpack('I', data_len_bytes)[0]
        except Exception:
            print("Connection lost (Recv Header)")
            break

        # 3. 영상 데이터 수신
        img_data = b''
        while len(img_data) < data_len:
            packet = client_cam.recv(data_len - len(img_data))
            if not packet: break
            img_data += packet
        
        if len(img_data) < data_len: break

        # 4. 이미지 디코딩 및 전처리
        np_data = np.frombuffer(img_data, dtype='uint8')
        frame = cv2.imdecode(np_data, 1)
        
        if frame is None: continue

        # 화면 출력용 리사이즈
        frame_show = cv2.resize(frame, (320, 240))
        cv2.imshow('AI Driving (Teachable Machine)', frame_show)

        # AI 입력용 전처리 (PIL 사용 - Teachable Machine 표준)
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
        
        image_array = np.asarray(image)
        # 정규화 (-1 ~ 1)
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1.0
        data[0] = normalized_image_array

        # 5. 추론 (Inference)
        prediction = model.predict(data, verbose=0)
        idx = np.argmax(prediction)
        
        # 결과 출력
        print(f"Pred: {names[idx]} ({prediction[0][idx]:.2f})")

        # 6. 모터 제어 신호 전송 (문자 방식)
        try:
            client_mot.sendall(cmd_chars[idx])
        except Exception:
            print("Motor Connection lost")
            break

        # 종료 키 (ESC)
        key = cv2.waitKey(1)
        if key == 27:
            break

        # FPS 출력
        cnt_frame += 1
        t_now = time.time()
        if t_now - t_prev > 1.0:
            print(f"FPS: {cnt_frame}")
            t_prev = t_now
            cnt_frame = 0

except KeyboardInterrupt:
    print("Stopping...")

finally:
    if client_cam: client_cam.close()
    if client_mot: client_mot.close()
    cv2.destroyAllWindows()

Checking required packages...
Connecting to ESP32-CAM at 192.168.0.60...
Connected successfully!
'keras_model.h5' not found. Please select a model file.
Checking compatibility for: C:/Users/119hw/Desktop/converted_keras/keras_model.h5
Loading Keras model...
Model loaded successfully!
Pred: _3_stop (0.57)
Pred: _0_forward (0.71)
Pred: _0_forward (0.65)
Pred: _0_forward (0.85)
Pred: _0_forward (0.77)
Pred: _0_forward (0.87)
Pred: _0_forward (0.68)
FPS: 7
Pred: _0_forward (0.65)
Pred: _0_forward (0.93)
Pred: _0_forward (0.68)
Pred: _0_forward (0.81)
Pred: _0_forward (0.81)
Pred: _0_forward (0.91)
Pred: _0_forward (0.87)
Pred: _0_forward (0.87)
Pred: _0_forward (0.92)
Pred: _0_forward (0.87)
Pred: _0_forward (0.74)
Pred: _0_forward (0.72)
Pred: _3_stop (0.63)
Pred: _0_forward (0.71)
FPS: 14
Pred: _0_forward (0.54)
Pred: _3_stop (0.65)
Pred: _0_forward (0.98)
Pred: _0_forward (0.60)
Pred: _0_forward (0.93)
Pred: _0_forward (0.96)
Pred: _0_forward (0.98)
Pred: _0_forward (0.93)
Pred: _0_forw

In [4]:
import sys
import subprocess
import importlib
import os
import time
import struct
import socket
import json # 설정 파일 로드용
import tkinter as tk
from tkinter import simpledialog, filedialog, messagebox

# ---------------------------------------------------------
# 1. 패키지 자동 설치
# ---------------------------------------------------------
def install_package(module_name, package_name=None):
    if package_name is None:
        package_name = module_name
    try:
        importlib.import_module(module_name)
    except ImportError:
        print(f"Installing {package_name} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

print("Checking required packages...")
install_package("h5py")
install_package("numpy")
install_package("cv2", "opencv-python")
install_package("PIL", "Pillow")
install_package("tensorflow")

import h5py
import cv2
import numpy as np
from PIL import Image, ImageOps
import tensorflow as tf
from tensorflow.keras.models import load_model

# ---------------------------------------------------------
# 2. 설정 및 연결
# ---------------------------------------------------------
HOST_CAM = '192.168.0.60' # [주의] 초기 IP
PORT_CAM = 80
PORT_MOT = 81

client_cam = None
client_mot = None

# Tkinter (IP 입력 및 파일 선택용)
root = tk.Tk()
root.withdraw() 

while True:
    try:
        print(f"Connecting to {HOST_CAM}...")
        client_cam = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_mot = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        # 반응 속도 최적화
        client_mot.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
        
        client_cam.settimeout(3)
        client_mot.settimeout(3)
        
        client_cam.connect((HOST_CAM, PORT_CAM))
        client_mot.connect((HOST_CAM, PORT_MOT))
        
        client_cam.settimeout(None)
        client_mot.settimeout(None)
        print("Connected successfully!")
        break 
    except Exception as e:
        print(f"Connection Failed: {e}")
        new_ip = simpledialog.askstring("Connection Failed", 
                                      f"Failed to connect to {HOST_CAM}.\n\nEnter ESP32 IP Address:",
                                      parent=root,
                                      initialvalue=HOST_CAM)
        if new_ip: HOST_CAM = new_ip
        else: sys.exit()

# ---------------------------------------------------------
# 3. 모델 로드 (Teachable Machine 호환성 패치 포함)
# ---------------------------------------------------------
model_filename = 'keras_model.h5'

# 파일 없으면 선택창
if not os.path.exists(model_filename):
    root.lift(); root.attributes('-topmost', True)
    model_filename = filedialog.askopenfilename(title="Select Teachable Machine Model", filetypes=[("H5 Files", "*.h5")])
    root.attributes('-topmost', False)
    
    if not model_filename:
        print("No model selected."); sys.exit()

# 모델 구조 패치 (groups=1 버그 수정)
try:
    f = h5py.File(model_filename, mode="r+")
    cfg = f.attrs.get("model_config")
    if isinstance(cfg, bytes): cfg = cfg.decode('utf-8')
    if '"groups": 1,' in cfg:
        print("Patching model config...")
        f.attrs.modify('model_config', cfg.replace('"groups": 1,', ''))
        f.flush()
    f.close()
except: pass

try:
    print(f"Loading model: {model_filename} ...")
    model = load_model(model_filename, compile=False)
    print("Model loaded.")
except Exception as e:
    messagebox.showerror("Error", f"Failed to load model:\n{e}")
    sys.exit()

root.destroy()

names = ['_0_forward', '_1_right', '_2_left', '_3_stop']
cmd_chars = [b'F', b'R', b'L', b'S'] 

# ---------------------------------------------------------
# 4. 마스킹 설정 로드 및 트랙바 초기화
# ---------------------------------------------------------
CONFIG_FILE = "mask_config.json"
crop_top, crop_bottom, crop_left, crop_right = 0, 0, 0, 0

# 설정 파일 로드
if os.path.exists(CONFIG_FILE):
    try:
        with open(CONFIG_FILE, 'r') as f:
            config = json.load(f)
            crop_top = config.get("top", 0)
            crop_bottom = config.get("bottom", 0)
            crop_left = config.get("left", 0)
            crop_right = config.get("right", 0)
        print(f"Loaded mask config: {config}")
    except:
        print("Failed to load mask config. Using default (0).")

def nothing(x): pass

WINDOW_NAME = 'Teachable Machine Driving (Masking Control)'
cv2.namedWindow(WINDOW_NAME)

# 트랙바 생성
cv2.createTrackbar('Top', WINDOW_NAME, crop_top, 240, nothing)
cv2.createTrackbar('Bottom', WINDOW_NAME, crop_bottom, 240, nothing)
cv2.createTrackbar('Left', WINDOW_NAME, crop_left, 320, nothing)
cv2.createTrackbar('Right', WINDOW_NAME, crop_right, 320, nothing)

# ---------------------------------------------------------
# 5. 주행 루프
# ---------------------------------------------------------
t_prev = time.time()
cnt_frame = 0
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# 데이터 수집 때와 해상도 통일
DISPLAY_WIDTH = 640
DISPLAY_HEIGHT = 480

try:
    while True:
        # 1. 영상 수신
        try:
            client_cam.sendall(struct.pack('B', 12))
            data_len_bytes = client_cam.recv(4)
            if not data_len_bytes: break
            data_len = struct.unpack('I', data_len_bytes)[0]
            
            img_data = b''
            while len(img_data) < data_len:
                packet = client_cam.recv(data_len - len(img_data))
                if not packet: break
                img_data += packet
            if len(img_data) < data_len: break
            
            np_data = np.frombuffer(img_data, dtype='uint8')
            frame = cv2.imdecode(np_data, 1)
            if frame is None: continue

        except Exception:
            print("Connection Error")
            break

        # 2. 리사이즈 (640x480) - 마스킹 기준 해상도
        frame_resized = cv2.resize(frame, (DISPLAY_WIDTH, DISPLAY_HEIGHT))

        # 3. 트랙바 값 읽어서 마스킹 적용
        c_top = cv2.getTrackbarPos('Top', WINDOW_NAME)
        c_bottom = cv2.getTrackbarPos('Bottom', WINDOW_NAME)
        c_left = cv2.getTrackbarPos('Left', WINDOW_NAME)
        c_right = cv2.getTrackbarPos('Right', WINDOW_NAME)

        h, w, _ = frame_resized.shape
        if c_top > 0: frame_resized[:c_top, :] = 0
        if c_bottom > 0: frame_resized[h-c_bottom:, :] = 0
        if c_left > 0: frame_resized[:, :c_left] = 0
        if c_right > 0: frame_resized[:, w-c_right:] = 0

        # 화면 출력
        cv2.imshow(WINDOW_NAME, frame_resized)

        # 4. AI 모델 입력용 전처리 (PIL 변환 -> 224x224 리사이즈 -> 정규화)
        # 중요: 마스킹된 이미지(frame_resized)를 사용해야 함
        image = Image.fromarray(cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB))
        image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
        
        normalized_image_array = (np.asarray(image).astype(np.float32) / 127.5) - 1.0
        data[0] = normalized_image_array

        # 5. 추론
        prediction = model.predict(data, verbose=0)
        idx = np.argmax(prediction)
        
        print(f"Pred: {names[idx]} ({prediction[0][idx]:.2f})")

        # 6. 모터 제어
        try:
            client_mot.sendall(cmd_chars[idx])
        except:
            pass

        key = cv2.waitKey(1)
        if key == 27: break

        cnt_frame += 1
        if time.time() - t_prev > 1.0:
            print(f"FPS: {cnt_frame}")
            t_prev = time.time()
            cnt_frame = 0

except KeyboardInterrupt:
    print("Stopping...")
finally:
    if client_cam: client_cam.close()
    if client_mot: client_mot.close()
    cv2.destroyAllWindows()

Checking required packages...
Connecting to 192.168.0.60...
Connected successfully!
Loading model: C:/Users/119hw/Desktop/converted_keras/keras_model.h5 ...
Model loaded.
Loaded mask config: {'top': 0, 'bottom': 0, 'left': 0, 'right': 0}
Pred: _0_forward (0.97)
Pred: _0_forward (0.99)
Pred: _0_forward (0.96)
Pred: _0_forward (0.97)
Pred: _0_forward (0.96)
Pred: _0_forward (0.96)
Pred: _0_forward (0.97)
Pred: _0_forward (0.98)
Pred: _0_forward (0.98)
FPS: 9
Pred: _0_forward (0.94)
Pred: _0_forward (0.94)
Pred: _0_forward (0.95)
Pred: _0_forward (0.98)
Pred: _0_forward (0.96)
Pred: _0_forward (0.98)
Pred: _0_forward (0.94)
Pred: _0_forward (0.94)
Pred: _0_forward (0.96)
Pred: _0_forward (0.95)
Pred: _0_forward (0.98)
Pred: _0_forward (0.94)
Pred: _0_forward (0.94)
Pred: _0_forward (0.99)
FPS: 14
Pred: _0_forward (0.91)
Pred: _0_forward (0.97)
Pred: _0_forward (0.97)
Pred: _0_forward (0.96)
Pred: _0_forward (0.97)
Pred: _0_forward (0.97)
Pred: _0_forward (0.97)
Pred: _0_forward (0.98)
Pre

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2570: error: (-27:Null pointer) NULL window: 'Teachable Machine Driving (Masking Control)' in function 'cvGetTrackbarPos'
